## OMP Tutorial

Notations
- `Y` :  Actual signal. `Y.shape -> (NUM_FEATURES, NUM_PATCHES)`
- `D` :  Dictionary. Basis vectors are stored. `D.shape -> (NUM_FEATURES, NUM_BASIS)`
- `X` :  Sparse representation. `X.shape -> (NUM_BASIS, NUM_PATCHES)`
- `E` :  Errors between Y and approximate signal. Initial value is `Y`. `E.shape -> (NUM_FEATURES, NUM_PATCHES)`
- `I` :  Absolute value of inner products given by `np.abs(np.inner(D.T, E))`. `I.shape -> (NUM_BASIS,)`
- `S` :  Set of index with the largest inner product at iteration `k`. Initial value is empty list `[]`.

References
- https://sparse-plex.readthedocs.io/en/latest/book/pursuit/omp/algorithm.html
- https://www.ieice.org/ess/sita/forum/article/2015/201512081915.pdf




In [1]:
import numpy as np
from scipy.optimize import least_squares

In [2]:
NUM_PATCHES = 1
NUM_FEATURES = 10
NUM_BASIS = 20
MAX_ITER = 10
THRESHOLD = 1e-5

In [3]:
Y = np.array([4.74342, -4.74342, 1.58114, -4.74342, -1.58114, 1.58114, -4.74342, -1.58114, -4.74342, -4.74342])

D = np.array([[-1, -1, -1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1],
              [ 1,  1,  1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1],
              [-1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1],
              [ 1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1, -1, -1, -1],
              [ 1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1, -1],
              [ 1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1,  1, -1,  1, -1,  1],
              [-1, -1,  1,  1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1,  1,  1],
              [ 1, -1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1],
              [-1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1],
              [ 1,  1,  1,  1, -1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1]]) / np.sqrt(10)
E = Y

S = []

def approximate_func(X, Y, D, S):
    for s in S:
        Y = Y - D[:, s] * X[s]
    return Y

In [4]:
for k in range(MAX_ITER):

    I = np.abs(np.inner(D.T, E))  # Compute absolute value of inner products
    S.append(I.argmax())          # Add maximum index of I
    ans = least_squares(approximate_func, np.zeros(NUM_BASIS), args=(Y, D, S))  # Solve least squares problem
    X = ans.x.astype(int)  # Sparse representation
    E = ans.fun            # Erros between Y and approximate signal

    print("\n")
    print(f"Iteration: {k}")
    print(f"X: {X}")
    print(f"np.abs(E).sum(): {np.abs(E).sum()}")
    
    if np.abs(E).sum() < THRESHOLD:
        break



Iteration: 0
X: [-7  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
np.abs(E).sum(): 15.178943999999968


Iteration: 1
X: [ 0  0  0  0  0  0 10  0  0  0  0  0 -5  0  0  0  0  0  0  0]
np.abs(E).sum(): 6.124324158918171e-09
